In [21]:
import sys
import numpy as np
from collections import Counter

In [83]:
import numpy as np
import argparse
parser = argparse.ArgumentParser(description="NML single experiment")
parser.add_argument("--N", type=int, default=10000, help="number of samples")
parser.add_argument("--m0", type=int, default=5, help="number of distinct values of the multinomial r.v X")
parser.add_argument("--m1", type=int, default=6, help="number of distinct values of the multinomial r.v Y")
args = parser.parse_args([])


x0 = np.random.randint(args.m0, size=args.N)
x1 = (x0 + np.random.randint(args.m1, size=args.N)) % args.m1

In [84]:
from __future__ import division
from math import ceil, log, sqrt
from scipy.stats import binom

def log2(n):
    return log(n or 1, 2)

def C_MN(n: int, K: int):
    """Computes the normalizing term of NML distribution recursively. O(n+K)
    
    For more detail, please refer to eq (19) (Theorem1) in 
    "NML Computation Algorithms for Tree-Structured Multinomial Bayesian Networks"
    https://pubmed.ncbi.nlm.nih.gov/18382603/
    
    and algorithm 2 in
    "Computing the Multinomial Stochastic Complexity in Sub-Linear Time"
    http://pgm08.cs.aau.dk/Papers/31_Paper.pdf
    
    
    Args
    ----------
        n (int): sample size of a dataset
        K (int): K-value multinomal distribution
        
    Returns
    ----------
        float: (Approximated) Multinomal Normalizing Sum 
    
    """
    
    total = 1
    b = 1
    d = 16 # 16 digit precision

    bound = int(ceil(2 + sqrt( -2 * n * np.log(2 * 10**(-d) - 100 ** (-d)))))

    for k in range(1, bound + 1):
        b = (n - k + 1) / n * b
        total += b

    old_sum = 1

    for j in range(3, K + 1):
        new_sum = total + (n * old_sum) / (j - 2)
        old_sum = total
        total = new_sum

    return total
    

def parametric_complexity(X, Y, model_type: str, X_ndistinct_vals=None, Y_ndistinct_vals=None):
    """Computes the Parametric Complexity of Multinomals. 
    
    Args
    ----------
        model_type (str): ["to", "gets", "indep", "confounder"]
        X (sequence): sequence of discrete outcomes
        Y (sequence): sequence of discrete outcomes
        X_ndistinct_vals (int): number of distinct values of the multinomial r.v X.
        Y_ndistinct_vals (int): number of distinct values of the multinomial r.v Y.
         
    Returns
    ----------
        float: Parametric Complexity of Multinomals 
        
    """
    
    assert len(X)==len(Y)
    n = len(X)
    X_ndistinct_vals = X_ndistinct_vals or len(np.unique(X))
    Y_ndistinct_vals = Y_ndistinct_vals or len(np.unique(Y))
    
    
    if model_type == "confounder":
        return  log2(C_MN(n=n, K=X_ndistinct_vals * Y_ndistinct_vals))
    
    else:
        return  log2(C_MN(n=n, K=X_ndistinct_vals)) + log2(C_MN(n=n, K=Y_ndistinct_vals))

In [85]:
%%time
C_MN(n=10000, K=20)

CPU times: user 442 µs, sys: 3 µs, total: 445 µs
Wall time: 454 µs


8.985119564280846e+29

In [86]:
%%time
C_MN(n=10000000, K=20)

CPU times: user 12.9 ms, sys: 6 µs, total: 12.9 ms
Wall time: 12.5 ms


2.1526114216272685e+58

In [87]:
parametric_complexity(x0, x1, model_type="to")

55.64200710898581

In [88]:
parametric_complexity(x0, x1, model_type="confounder")

143.42425858956648

In [89]:
%%time
parametric_complexity(x0, x1, model_type="to")

CPU times: user 3.99 ms, sys: 2 ms, total: 5.99 ms
Wall time: 3.6 ms


55.64200710898581

In [90]:
parametric_complexity(x0, x1, model_type="gets")

55.64200710898581

In [91]:
%%time
parametric_complexity(x0, x1, model_type="confounder")

CPU times: user 4.09 ms, sys: 1.01 ms, total: 5.1 ms
Wall time: 3.38 ms


143.42425858956648

In [92]:
X=np.random.randint(5, size=100000)
Y=np.random.randint(4, size=100000)

In [93]:
f = map_to_majority(X, Y)

In [106]:
f

{3: 0, 2: 2, 4: 2, 0: 3, 1: 1}

In [105]:
f[3]= 0

In [208]:
sys.float_info.max

1.7976931348623157e+308

In [212]:
# 全探索するバージョン　主にデバック用
def update_regression(X, Y, f=None):
    supp_X = sorted(list(set(X)))
    suppY = sorted(list(set(Y)))
    best_ml = sys.float_info.max
    ans = None
    for ys in itertools.product(suppY, repeat=len(supp_X)):
        f = dict()
        for i, x in enumerate(supp_X):
            f[x] = ys[i]
        print(f)

        temp_ml = cause_effect_negloglikelihood(X, Y, f)

        if temp_ml < best_ml:
            ans = f
            best_ml = temp_ml
    return ans

In [210]:
ans 
best_ml

432174.036394854

In [215]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
# ref: https://github.molgen.mpg.de/EDA/cisc/blob/master/formatter.py
"""This module provides common methods for manipulating data"""
from collections import defaultdict


def stratify(X, Y):
    """Stratifies Y based on unique values of X.
    Args:
        X (sequence): sequence of discrete outcomes
        Y (sequence): sequence of discrete outcomes
    Returns:
        (dict): list of Y-values for a X-value
    """
    Y_grps = defaultdict(list)
    for i, x in enumerate(X):
        Y_grps[x].append(Y[i])
    return Y_grps

In [216]:
def map_to_majority(X, Y):
    """Creates a function that maps x to most frequent y.
    Args:
        X (sequence): sequence of discrete outcomes
        Y (sequence): sequence of discrete outcomes
    Returns:
        (dict): map from Y-values to frequently co-occuring X-values
    """
    f = dict()
    Y_grps = stratify(X, Y)
    for x, Ys in Y_grps.items():
        frequent_y, _ = Counter(Ys).most_common(1)[0]
        f[x] = frequent_y
    return f

In [217]:
def update_regression(C, E, f, max_niterations=10000):
    """Update discrete regression with C as a cause variable and Y as a effect variable
    so that it maximize likelihood
    Args
    -------
        C (sequence): sequence of discrete outcomes
        E (sequence): sequence of discrete outcomes
        f (dict): map from C to Y
        
    """
    cur_likelihood =  cause_effect_negloglikelihood(C, E, f)
    supp_C = list(set(C))
    supp_E = list(set(E))
    
    j = 0
    minimized = True
    while j < max_niterations and minimized:
        minimized = False
        
        for c_to_map in supp_C:
            best_likelihood = sys.float_info.max
            best_e = None
            
            for cand_e in supp_E:
#                 if cand_e == f[c_to_map]:
#                     continue
                
                f_ = dict()
                for c_value, e_value in f.items():
                    if c_value == c_to_map:
                        f_[c_value] = cand_e
                    else:
                        f_[c_value] = e_value
                        
                neglikelihood = cause_effect_negloglikelihood(C, E, f_)
                
                if neglikelihood < best_likelihood:
                    best_likelihood = neglikelihood
                    best_e = cand_e

            if best_likelihood < cur_likelihood:
                #print(best_likelihood, cur_likelihood, f)
                cur_likelihood = best_likelihood
                f[c_to_map] = best_e
                minimized = True
        j += 1
    
    return f
        

In [218]:
def cause_effect_negloglikelihood(C, E, func):
    """Compute negative log likelihood for cause & effect pair.
    Model type : C→E
    
    Args
    -------
        C (sequence): sequence of discrete outcomes (Cause)
        E (sequence): sequence of discrete outcomes (Effect)
        func (dict): map from C-value to E-value
    
    Returns
    -------
        (float): maximum log likelihood 
    """
    supp_C = list(set(C))
    supp_E = list(set(E))
    mod_C = len(set(C))
    mod_E = len(set(E))
    C_freqs = Counter(C)
    E_freqs = Counter(E)
    n = len(C)
    
    pair_cnt = defaultdict(lambda: defaultdict(int))
    for c, e in zip(C, E):
        pair_cnt[c][e] += 1
        
    loglikelihood = 0
    
    for freq in C_freqs.values():
        loglikelihood += freq * (log2(n) - log2(freq))            

    for e_E in supp_E:
        freq = 0
        for e in supp_E:
            for c in supp_C:
                if (e - func[c]) % mod_E == e_E:
                    freq += pair_cnt[c][e]
        loglikelihood += freq * (log2(n) - log2(freq))
        
    return loglikelihood

def neg_log_likelihood(X, Y, model_type):
    """Compute negative maximum log-likelihood of the model given observations z^n.
    
    Args
    ------
        X (sequence): sequence of discrete outcomes
        Y (sequence): sequence of discrete outcomes
        model_type (str): one of ["to", "gets", "indep", "confounder"]
        f (dict): map from Y-values to frequently co-occuring X-values
    Returns
    -----------
        (float): (negative) maximum log likelihood 
    """

    n = len(X)
    loglikelihood = 0
            
    if model_type == "to":
        f = map_to_majority(X, Y)
        f = update_regression(X, Y, f)
        loglikelihood += cause_effect_negloglikelihood(X, Y, f)
                
    elif model_type == "gets":
        g = map_to_majority(Y, X)
        g = update_regression(Y, X, g)
        loglikelihood += cause_effect_negloglikelihood(Y, X, g)

    elif model_type == "indep":
        X_freqs = Counter(X)
        Y_freqs = Counter(Y)
        for freq in X_freqs.values():
            loglikelihood += freq * (log2(n) - log2(freq))  
        for freq in Y_freqs.values():
            loglikelihood += freq * (log2(n) - log2(freq)) 
    
    elif model_type == "confounder":
        pair_cnt = defaultdict(lambda: defaultdict(int))
        for x, y in zip(X, Y):
            pair_cnt[x][y] += 1

        for x in list(set(X)):
            for y in list(setY):
                loglikelihood += pair_cnt[x][y] * (log2(n) - log2(pair_cnt[x][y]))
    
    return loglikelihood
        

In [219]:
neg_log_likelihood(x0, x1, model_type="to")

49060.172312384035

In [220]:
neg_log_likelihood(x0, x1, model_type="gets")

49057.390946689746

In [190]:
neg_log_likelihood(x0, x1, model_type="indep")

49063.66126947885

In [191]:
neg_log_likelihood(x0, x1, model_type="confounder")

49053.186994119234

In [158]:
neg_log_likelihood(X=np.random.randint(5, size=100000),
                          Y=np.random.randint(4, size=100000), model_type="to")

432185.64731907286

In [159]:
neg_log_likelihood(X=np.random.randint(5, size=100000),
                          Y=np.random.randint(4, size=100000), model_type="gets")

432183.674321685

In [160]:
neg_log_likelihood(X=np.random.randint(5, size=100000),
                          Y=np.random.randint(4, size=100000), model_type="indep")

432191.1058337093

In [161]:
neg_log_likelihood(X=np.random.randint(5, size=100000),
                          Y=np.random.randint(4, size=100000), model_type="confounder")

432174.0353961935

In [223]:
def sc(X, Y, model_type: str, X_ndistinct_vals=None, Y_ndistinct_vals=None):
    """Computes the stochastic complexity of z^n(two discrete sequences).
    
    Args
    ------        
        X (sequence): sequence of discrete outcomes
        Y (sequence): sequence of discrete outcomes
        model_type (str): ["to", "gets", "indep", "confounder"]
        X_ndistinct_vals (int): number of distinct values of the multinomial r.v X.
        Y_ndistinct_vals (int): number of distinct values of the multinomial r.v Y.
         
    Returns
    ----------
        float: Stochastic Complexity of a given dataset
    """
    assert len(X)==len(Y)
    X_ndistinct_vals = X_ndistinct_vals or len(set(X))
    Y_ndistinct_vals = Y_ndistinct_vals or len(set(Y))
    
    negloglikelihood =  neg_log_likelihood(X, Y, model_type)
    param_comp = parametric_complexity(X, Y, model_type, X_ndistinct_vals, Y_ndistinct_vals)
    
    stochastic_complexity = negloglikelihood + param_comp
    
    # add function code length
    if model_type == "to":
        stochastic_complexity += X_ndistinct_vals * log2(Y_ndistinct_vals)
    elif model_type == "gets":
        stochastic_complexity += Y_ndistinct_vals * log2(X_ndistinct_vals)
    
    return stochastic_complexity

In [224]:
MODEL_CANDIDATES = ["to", "gets", "indep", "confounder"]

def ndm(X, Y, ):
    """NML Discrete Model
    """
    
    results = []
    
    for model_type in MODEL_CANDIDATES:
        stochastic_complexity = sc(X, Y, model_type)
        results.append((stochastic_complexity, model_type))
    
    return results
    

In [225]:
ndm(x0, x1)

[(49115.81431949302, 'to'),
 (49113.032953798735, 'gets'),
 (49119.30327658784, 'indep'),
 (49196.611252708804, 'confounder')]

In [64]:
results = ndm(x0, x1)

In [65]:
results

[(49117.16378975504, 'to'),
 (49125.187882422644, 'gets'),
 (49119.35536012633, 'indep'),
 (49183.17050282375, 'confounder')]

In [66]:
results.sort(key=lambda x: x[0])

In [67]:
results

[(49117.16378975504, 'to'),
 (49119.35536012633, 'indep'),
 (49125.187882422644, 'gets'),
 (49183.17050282375, 'confounder')]

In [53]:
c = np.random.randint(args.m0, size=args.N)
x0 = (c + np.random.randint(args.m0, size=args.N)) % args.m0
x1 = (c + np.random.randint(args.m1, size=args.N)) % args.m1

In [149]:
x0 = np.random.randint(args.m0, size=args.N)
x1 = np.random.randint(args.m1, size=args.N)

In [209]:
x0 = np.random.randint(args.m0, size=args.N)
x1 = (x0 + np.random.randint(args.m1, size=args.N)) % args.m1

In [61]:
args.N=1000

In [165]:
from tqdm.notebook import tqdm
N = 10
ans = 0
for _ in tqdm(range(N)):
    x0 = np.random.randint(args.m0, size=args.N)
    x1 = (x0 + np.random.randint(args.m1, size=args.N)) % args.m1
    
    results = ndm(x0, x1)
    results.sort(key=lambda x: x[0])
    print(results)
    if results[0][1]=="to" or results[0][1]=="indep":
        ans += 1
print(ans/N)
    
    

  0%|          | 0/10 [00:00<?, ?it/s]

[(49115.78619862696, 'indep'), (49119.945181586125, 'to'), (49120.529461959355, 'gets'), (49183.94414821583, 'confounder')]
[(49115.089035500714, 'to'), (49115.471982056944, 'indep'), (49119.90202528516, 'gets'), (49183.00954554714, 'confounder')]
[(49116.68352184167, 'indep'), (49124.95796595048, 'to'), (49128.64011178271, 'gets'), (49195.515073452414, 'confounder')]
[(49119.23193351236, 'indep'), (49127.25551063263, 'to'), (49128.284969073655, 'gets'), (49197.60362497523, 'confounder')]
[(49120.24583773402, 'gets'), (49120.29925049768, 'indep'), (49121.01703279927, 'to'), (49184.85211677031, 'confounder')]
[(49123.44834688958, 'indep'), (49126.86807075386, 'gets'), (49127.07057326301, 'to'), (49194.93571532733, 'confounder')]
[(49115.91515990417, 'indep'), (49120.31611869461, 'to'), (49124.94196854635, 'gets'), (49191.67847162867, 'confounder')]
[(49117.67280524488, 'indep'), (49122.55295891437, 'to'), (49125.451311221215, 'gets'), (49191.232753503784, 'confounder')]
[(49118.53354468

In [50]:
args.N

10000

In [61]:
from tqdm.notebook import tqdm
N = 10
ans = 0
for _ in tqdm(range(N)):
    c = np.random.randint(args.m0 + 1, size=args.N)
    x0 = (c + np.random.randint(args.m0, size=args.N)) % args.m0
    x1 = (c + np.random.randint(args.m1, size=args.N)) % args.m1
    
    results = ndm(x0, x1)
    results.sort(key=lambda x: x[0])
    print(results)
    if results[0][1]=="to" or results[0][1]=="indep":
        ans += 1
print(ans/N)
    
    

  0%|          | 0/10 [00:00<?, ?it/s]

[(49122.13674379872, 'indep'), (49131.50511157087, 'to'), (49132.783806909996, 'gets'), (49200.69620523089, 'confounder')]
[(49120.11738581671, 'indep'), (49124.02923254763, 'to'), (49128.75407603558, 'gets'), (49187.607217876175, 'confounder')]
[(49120.8144659516, 'indep'), (49125.603028836296, 'to'), (49130.35314656131, 'gets'), (49193.850702007134, 'confounder')]
[(49121.76574651071, 'indep'), (49129.77783313896, 'to'), (49130.294178524564, 'gets'), (49194.99894835574, 'confounder')]
[(49118.92240430878, 'indep'), (49124.3913448307, 'to'), (49126.86498487197, 'gets'), (49183.45234864786, 'confounder')]
[(49112.75703222564, 'indep'), (49118.43681293126, 'gets'), (49120.665148048414, 'to'), (49182.3503248927, 'confounder')]
[(49119.44164044491, 'indep'), (49123.820828253265, 'to'), (49124.2101248229, 'gets'), (49193.42478849901, 'confounder')]
[(49122.30466170439, 'indep'), (49128.09815090055, 'to'), (49128.260309712015, 'gets'), (49196.594002018384, 'confounder')]
[(49121.55901332457